# EcoPoints Data Analysis

## Import needed libraries

In [1]:
import json
import pandas as pd
import pydeck as pdk
import plotly.express as px
import plotly.graph_objects as go
import plotly.io as pio
pio.renderers.default = 'notebook'
import plotly.offline as pyo
pyo.init_notebook_mode(connected=True)

from dash import Dash, dcc, html, Input, Output

In [2]:
with open('data/data.json') as f:
    data = json.load(f)

## Data Preperation

In [3]:
timestamps = []
engine_load = []
engine_rpm = []
coolant_temp = []
altitude = []
gps_speed = []
vehicle_speed = []
armin_speed = []
latitude = []
longitude = []

for entry in data:
    timestamps.append(pd.to_datetime(entry.get('timestamp'), unit='ms'))
    
    entry_data = entry.get('data', {})
    engine_load.append(float(entry_data.get('Engine Load', '0').replace(',', '.')))
    engine_rpm.append(float(entry_data.get('Engine RPM', '0').replace(',', '.')))
    coolant_temp.append(float(entry_data.get('Engine Coolant Temperature', '0').replace(',', '.')))
    altitude.append(float(entry_data.get('Altitude', '0').replace(',', '.')))
    gps_speed.append(float(entry_data.get('Gps-Speed', '0').replace(',', '.')))
    vehicle_speed.append(float(entry_data.get('Vehicle Speed', '0').replace(',', '.')))
    armin_speed.append(float(entry_data.get('Armin-Speed', '0').replace(',', '.')))
    latitude.append(float(entry_data.get('Latitude', '0')))
    longitude.append(float(entry_data.get('Longitude', '0')))

df = pd.DataFrame({
    'timestamp': timestamps,
    'Engine Load': engine_load,
    'Engine RPM': engine_rpm,
    'Coolant Temperature': coolant_temp,
    'Altitude': altitude,
    'Gps Speed': gps_speed,
    'Vehicle Speed': vehicle_speed,
    'Armin Speed': armin_speed,
    'Latitude': latitude,
    'Longitude': longitude
})

df_normalized = df.copy() 
columns_to_normalize = ['Engine Load', 'Engine RPM', 'Coolant Temperature', 'Altitude', 'Gps Speed', 'Vehicle Speed', 'Armin Speed']

for col in columns_to_normalize:
    df_normalized[col] = (df[col] - df[col].min()) / (df[col].max() - df[col].min())


In [4]:
df = df[:-1]  
df_normalized = df_normalized[:-1]

In [5]:
print(df.head())

                timestamp  Engine Load  Engine RPM  Coolant Temperature  \
0 2024-10-25 04:56:04.831          0.0      2079.0                 34.0   
1 2024-10-25 04:56:08.456          0.0      2114.0                 34.0   
2 2024-10-25 04:56:12.088          0.0      2092.0                 35.0   
3 2024-10-25 04:56:15.718         30.6      2069.0                 35.0   
4 2024-10-25 04:56:19.339          0.0      2210.0                 35.0   

     Altitude  Gps Speed  Vehicle Speed  Armin Speed   Latitude  Longitude  
0  482.705767       52.0           51.0          0.0  48.355464  14.181740  
1  473.135386       51.0           51.0          0.0  48.355013  14.181358  
2  467.637504       51.0           50.0          0.0  48.354500  14.181278  
3  464.523973       51.0           50.0          0.0  48.354135  14.181484  
4  459.997381       53.0           50.0          0.0  48.353731  14.182042  


In [6]:
print(df_normalized.head())

                timestamp  Engine Load  Engine RPM  Coolant Temperature  \
0 2024-10-25 04:56:04.831     0.000000    0.736190             0.478873   
1 2024-10-25 04:56:08.456     0.000000    0.748584             0.478873   
2 2024-10-25 04:56:12.088     0.000000    0.740793             0.492958   
3 2024-10-25 04:56:15.718     0.307229    0.732649             0.492958   
4 2024-10-25 04:56:19.339     0.000000    0.782578             0.492958   

   Altitude  Gps Speed  Vehicle Speed  Armin Speed   Latitude  Longitude  
0  1.000000   0.472727       0.463636          0.0  48.355464  14.181740  
1  0.980173   0.463636       0.463636          0.0  48.355013  14.181358  
2  0.968784   0.463636       0.454545          0.0  48.354500  14.181278  
3  0.962334   0.463636       0.454545          0.0  48.354135  14.181484  
4  0.952956   0.481818       0.454545          0.0  48.353731  14.182042  


## Driven Path

In [7]:
path_data = [
    {"path": [[row['Longitude'], row['Latitude']] for _, row in df.iterrows()]} 
]
layer = pdk.Layer(
    "PathLayer",
    path_data,
    get_path='path',
    get_width=1,  
    get_color='[0, 128, 255]',  
    width_scale=10,
    pickable=True,
)
point_layer = pdk.Layer(
    "ColumnLayer",
    df,
    get_position='[Longitude, Latitude]',
    get_elevation='Altitude',
    elevation_scale=0.1, 
    radius=5,
    get_fill_color='[255, 0, 0]',
    pickable=True,
)
view_state = pdk.ViewState(
    latitude=df['Latitude'].mean() if not df['Latitude'].empty else 0.0, 
    longitude=df['Longitude'].mean() if not df['Longitude'].empty else 0.0, 
    zoom=12, 
    bearing=0,
    pitch=0,
)

deck = pdk.Deck(layers=[layer], initial_view_state=view_state)
deck.to_html("html_files/driven_route_map.html")

## Driven Altitude

In [17]:
path_data = [{"path": df[['Longitude', 'Latitude', 'Altitude']].values.tolist()}]

# Define a PathLayer with fixed color to simplify debugging
path_layer = pdk.Layer(
    "PathLayer",
    path_data,
    pickable=True,
    get_path="path",
    get_width=5,
    get_color=[255, 0, 0, 200],  # Fixed red color for simplicity
    width_scale=10,
    width_min_pixels=1,
    auto_highlight=True,
)

# Initial view state centered on the data
initial_view_state = pdk.ViewState(
    latitude=df['Latitude'].mean(),
    longitude=df['Longitude'].mean(),
    zoom=10,
    pitch=50,
)

# Render the map
r = pdk.Deck(layers=[path_layer], initial_view_state=initial_view_state)
r.to_html("vehicle_path.html")

## All Car Metrics in one

### Original Values

In [9]:
carMetrOrig = Dash(__name__)

carMetrOrig.layout = html.Div([
    html.H4('Vehicle Data Over Time'),
    dcc.Graph(id="graph", style={'height': '700px'}),
    dcc.Checklist(
        id="checklist",
        options=[
            {'label': 'Engine Load', 'value': 'Engine Load'},
            {'label': 'Engine RPM', 'value': 'Engine RPM'},
            {'label': 'Coolant Temperature', 'value': 'Coolant Temperature'},
            {'label': 'Altitude', 'value': 'Altitude'},
            {'label': 'Gps Speed', 'value': 'Gps Speed'},
            {'label': 'Vehicle Speed', 'value': 'Vehicle Speed'},
            {'label': 'Armin Speed', 'value': 'Armin Speed'}
        ],
        value=['Engine RPM'],
        inline=True
    ),
])

@carMetrOrig.callback(
    Output("graph", "figure"), 
    Input("checklist", "value"))
def update_line_chart(selected_metrics):
    fig = px.line(df, x='timestamp', y=selected_metrics)
    
    fig.update_layout(
        title="Selected Vehicle Metrics Over Time",
        xaxis_title="Timestamp",
        yaxis_title="Value",
        legend_title="Metrics",
        xaxis={
            'rangeslider': {'visible': True},
            'type': 'date', 
            'range': [df['timestamp'].min(), df['timestamp'].max()]
        }
    )
    return fig

if __name__ == '__main__':
    carMetrOrig.run_server(debug=True, port=8050)


### Normalized Values

In [10]:
carMetrNorm = Dash(__name__)

carMetrNorm.layout = html.Div([
    html.H4('Normalized Vehicle Data Over Time'),
    dcc.Graph(id="graph", style={'height': '700px'}),
    dcc.Checklist(
        id="checklist",
        options=[
            {'label': 'No. Engine Load', 'value': 'Engine Load'},
            {'label': 'No. Engine RPM', 'value': 'Engine RPM'},
            {'label': 'No. Coolant Temperature', 'value': 'Coolant Temperature'},
            {'label': 'No. Altitude', 'value': 'Altitude'},
            {'label': 'No. Gps Speed', 'value': 'Gps Speed'},
            {'label': 'No. Vehicle Speed', 'value': 'Vehicle Speed'},
            {'label': 'No. Armin Speed', 'value': 'Armin Speed'}
        ],
        value=['Engine RPM'],
        inline=True
    ),
])

@carMetrNorm.callback(
    Output("graph", "figure"), 
    Input("checklist", "value"))
def update_line_chart(selected_metrics):
    fig = px.line(df_normalized, x='timestamp', y=selected_metrics)
    
    fig.update_layout(
        title="Selected Normalized Vehicle Metrics Over Time",
        xaxis_title="Timestamp",
        yaxis_title="Normalized Value",
        legend_title="Metrics",
        xaxis={
            'rangeslider': {'visible': True},
            'type': 'date', 
            'range': [df['timestamp'].min(), df['timestamp'].max()] 
        }
    )
    return fig

if __name__ == '__main__':
    carMetrNorm.run_server(debug=True, port=8051)

## Altitude and Engine Load (Normalized)

In [11]:
df['Normalized Altitude'] = (df['Altitude'] - df['Altitude'].min()) / (df['Altitude'].max() - df['Altitude'].min())
df['Normalized Engine Load'] = (df['Engine Load'] - df['Engine Load'].min()) / (df['Engine Load'].max() - df['Engine Load'].min())

app = Dash(__name__)

app.layout = html.Div([
    html.H4('Normalized Altitude and Engine Load Over Time'),
    dcc.Graph(id="graph", style={'height': '700px'})
])

@app.callback(
    Output("graph", "figure"), 
    Input("graph", "id") 
)
def update_line_chart(_):
    fig = go.Figure()

    fig.add_trace(go.Scatter(
        x=df['timestamp'],
        y=df['Normalized Altitude'],
        mode='lines',
        name='Normalized Altitude',
        line=dict(color='orange', width=2),
        marker=dict(size=5, symbol='circle'),
    ))

    fig.add_trace(go.Scatter(
        x=df['timestamp'],
        y=df['Normalized Engine Load'],
        mode='lines',
        name='Normalized Engine Load',
        line=dict(color='blue', width=2),
        marker=dict(size=5, symbol='circle'),
    ))

    fig.update_layout(
        title='Normalized Altitude and Engine Load Over Time',
        xaxis_title='Timestamp',
        yaxis_title='Normalized Values',
        legend_title='Legend',
        xaxis=dict(
            showgrid=True,
            rangeslider=dict(visible=True),
            type='date',
            range=[df['timestamp'].min(), df['timestamp'].max()]
        ),
        yaxis=dict(showgrid=True),
        hovermode='x unified'
    )
    
    return fig

if __name__ == '__main__':
    app.run_server(debug=True, port=8052)